# Analyse of Grenoble Documents

Analysis of the PLU of Grenoble. First step

## Imports

In [ ]:
import json
from pathlib import Path

import yaml
from google.genai import types

from src.api.gemini_thinking import gemini_api
from src.config import (
    INTERIM_DATA_DIR,
    PROJ_ROOT,
    RAW_DATA_DIR,
)

# Load pompts and tree data
with open(PROJ_ROOT / "config/prompts.json", "r", encoding="utf-8") as f:
    prompts = json.load(f)

with open(PROJ_ROOT / "config/plu_tree.yaml", "r", encoding="utf-8") as f:
    tree_grenoble = yaml.safe_load(f)["Grenoble"]

## Functions

In [ ]:
def retrieve_zone_pages(ocr_json: dict, prompt: str) -> dict:
    """
    Récupère les pages du document et les affiche dans des parties séparées.

    Args:
        ocr_json: (dict) La réponse JSON brute de l'OCR.

    Returns:
        La réponse brute complète du modèle sous format dict:
        {
        "Zone1": [liste des numéros de pages avec informations sur la zone Zone1],
        "Zone2": [liste des numéros de pages avec informations sur la zone Zone2],
        "Zone3": [liste des numéros de pages avec informations sur la zone Zone3],
        "...": [...],
        }
    """
    instruction: str = types.Part.from_text(text=prompt)
    parts = [instruction]
    for i, page in enumerate(ocr_json.get("pages")):
        parts.append(
            types.Part.from_text(
                text=json.dumps(
                    f"PAGE {i + 1} : " + page["markdown"], ensure_ascii=False
                )
            )
        )
    response = gemini_api(parts=parts)
    return response

## Prompts

### Constants

In [ ]:
raw_dir = RAW_DATA_DIR
int_dir = INTERIM_DATA_DIR
folder = "Grenoble"

### Retrieve Data

**Résultats**
- documents_generaux: `{nom_document: contenu_json}`
    
    Un dictionnaire où les clés sont les noms des documents généraux et les valeurs sont
    leur contenu au format JSON.

- data_par_zone: `{type_zone: {nom_zone: [pages_texte]}}`
    
    Un dictionnaire où les clés sont les noms des zones et les valeurs sont des sous-dictionnaires
    où chaque clé représente une zone spécifique *(ex: 'Zone_A', 'Zone_AL')* avec comme valeurs
    les listes des pages extraites du document original correspondant à cette zone.

- data_zone: `{type_zone: {nom_zone: contenu_json}}`
    
    Un dictionnaire imbriqué où le premier niveau représente les types de zones
    *(ex: 'Zone Urbaine', 'Zone Agricole')* et le second niveau contient les données
    spécifiques à chaque zone où les clés sont les noms des zones *(ex: 'Zone_A')*
    et les valeurs sont leur contenu au format JSON.

#### Documents Généraux

In [ ]:
data_generaux = {}
for doc in tree_grenoble["documents_generaux"]:
    path_doc = Path(raw_dir / Path("Grenoble") / doc).with_suffix(".json")
    with open(path_doc, "r", encoding="utf-8") as f:
        data_generaux[doc] = json.load(f)

print(data_generaux.keys())

#### Documents Zones

In [ ]:
data_zone_unique = {}
for zone_a, zones in tree_grenoble["documents_par_zone"].items():
    data_zone_unique[zone_a] = {}
    for zone in zones:
        path_zone = Path(raw_dir / Path("Grenoble") / zone_a / zone).with_suffix(
            ".json"
        )
        with open(path_zone, "r", encoding="utf-8") as f:
            data_zone_unique[zone_a][zone] = json.load(f)

print(data_zone_unique.keys())

#### Documents Par Zone

**On récupère d'abord les pages de la zone.**

In [ ]:
pages_dict = {}
["Zone à Urbaniser", "Zone Agricoles", "Zone Dédiée", "Zone Urbaines"]

In [ ]:
zone_a = "Zone Urbaines"

par_zone = "Par " + zone_a
path_par_zone = Path(raw_dir / Path("Grenoble") / par_zone).with_suffix(".json")
with open(path_par_zone, "r", encoding="utf-8") as f:
    data_zone = json.load(f)
response = retrieve_zone_pages(ocr_json=data_zone, prompt=prompts["retrieve_page_zone"])
pages_dict[zone_a] = json.loads(response.text.replace("```", ""))

**On isole les pages à partir de là.**

In [ ]:
data_par_zone = {}
for zone_a in tree_grenoble["documents_par_zone"]:
    par_zone = "Par " + zone_a
    path_par_zone = Path(raw_dir / Path("Grenoble") / par_zone).with_suffix(".json")
    with open(path_par_zone, "r", encoding="utf-8") as f:
        data_zone = json.load(f)

    data_par_zone[zone_a] = {}
    for zone, pages in pages_dict[zone_a].items():
        data_par_zone[zone_a][zone] = [data_zone["pages"][i] for i in pages]

#### Formattage vant de sauvegarder

Template

In [ ]:
mapping = {
    "ZoneUA1": "Zone_UA1",
    "ZoneUA2": "Zone_UA2",
    "ZoneUA3": "Zone_UA3",
    # ...,
    "informations_generales": "informations_generales",
}

mon_dict = dict()

# Ou pour modifier le dictionnaire existant
for old_key, new_key in mapping.items():
    if old_key in mon_dict:
        mon_dict[new_key] = mon_dict.pop(old_key)

**SAVE THE RESULTS**

In [ ]:
with open("output/data_par_zone.json", "w", encoding="utf-8") as f:
    json.dump(data_par_zone, f, ensure_ascii=False, indent=4)

with open("output/data_zone_unique.json", "w", encoding="utf-8") as f:
    json.dump(data_zone_unique, f, ensure_ascii=False, indent=4)

with open("output/data_generaux.json", "w", encoding="utf-8") as f:
    json.dump(data_generaux, f, ensure_ascii=False, indent=4)

### Création des prompts

In [ ]:
def get_prompt_grenoble(
    zone: str,
    prompt: str,
) -> str:
    """
    Récupère le prompt pour la ville de Grenoble.

    Args:
        data_general: (dict) Les données générales de la ville de Grenoble.
        data_par_zone: (dict) Les données par zone de la ville de Grenoble.
        data_zone: (dict) Les données de zone de la ville de Grenoble.
        type_zone: (str) Le type de zone à analyser.
        zone: (str) La zone à analyser.
        prompt: (str) Le prompt de base.

    Returns:
        str: Le prompt pour la ville de Grenoble.
    """
    return prompt.format(
        ZONE_CADASTRALE_CIBLE=zone,
    )

In [ ]:
folder_dir: Path = int_dir / folder
folder_dir.mkdir(exist_ok=True)

# On ne garde que les données textuelles des Dispositions Générales
temp_data = data_generaux["Dispositions Générales"]["pages"]
temp_data_generaux = [
    f"page {temp_data[i]['index'] + 1}: " + temp_data[i]["markdown"]
    for i in range(len(temp_data))
]

for par_zone in tree_grenoble["documents_par_zone"]:
    par_zone_dir: Path = folder_dir / par_zone
    par_zone_dir.mkdir(exist_ok=True)

    for zone in tree_grenoble["documents_par_zone"][par_zone]:
        # Ce sont les données textuelles des PLU par type de zone filtrées
        temp_data = data_par_zone[par_zone][zone]
        temp_data_par_zone = [
            f"page {temp_data[i]['index'] + 1}: " + temp_data[i]["markdown"]
            for i in range(len(temp_data))
        ]

        # Ce sont les données textuelles uniquement des PLU de chaque Zone
        temp_data = data_zone_unique[par_zone][zone]["pages"]
        temp_data_zone_unique = [
            f"page {temp_data[i]['index'] + 1}: " + temp_data[i]["markdown"]
            for i in range(len(temp_data))
        ]
        document_content = {
            f"PLU de la {zone.replace('_', ' ')}": temp_data_zone_unique,
            f"Réglement des {par_zone}": temp_data_par_zone,
            "Dispositions Générales": temp_data_generaux,
        }
        save_folder = Path(f"output/{par_zone}")
        save_folder.mkdir(exist_ok=True, parents=True)
        save_path = save_folder / Path(zone).with_suffix(".json")
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(document_content, fp=f, indent=4, ensure_ascii=False)


## Analyse

### Test Case : Zone_AU

Il me faut le `prompt` : "data/interim/Grenoble/prompts/Zone à Urbaniser/Zone_AU.txt"

Et les images qui vont avec, à convertir en `types.format_text` : 
- data/raw/markdown/Grenoble/Zone à Urbaniser/Zone_AU/images
- data/raw/markdown/Grenoble/Par Zone à Urbaniser/images
- data/raw/markdown/Grenoble/Dispositions Générales/images

En triant les images qui sont utiles et à insérer.

In [ ]:
import re
from PIL import Image
import os

with open("output/Zone Urbaines/Zone_UC2.json", "r", encoding="utf-8") as f:
    data_zone_uc2 = json.load(f)

image_folder_dg = (
    "/mnt/mydisk/Projects/plu/data/raw/markdown/Grenoble/Dispositions Générales/images"
)
image_folder_zone_uc = (
    "/mnt/mydisk/Projects/plu/data/raw/markdown/Grenoble/Zone Urbaine/Zone_UC2/images"
)
image_folder_par_zone = (
    "/mnt/mydisk/Projects/plu/data/raw/markdown/Grenoble/Par Zone Urbaines/images"
)

pattern = r"!\[(?P<alt>[^\]]*)\]\((?P<filename>[^)]+)\)"

In [ ]:
from pathlib import Path
import re


def text_to_part_with_images(text: str, pattern: str, image_folder: Path) -> list:
    """
    Scinde une chaîne de texte selon un pattern donné.
    Retourne une liste contenant les parties de texte et les éléments correspondant au pattern.

    Args:
        text (str): Le texte à analyser.
        pattern (str): Le motif regex pour identifier les images.
        image_folder (Path): Le dossier contenant les images.

    Returns:
        list: Une liste d'objets Part contenant le texte et les images.
    """
    # Compile le motif regex pour une meilleure performance
    regex = re.compile(pattern)

    # Trouve tous les matches du pattern dans le texte
    matches = list(regex.finditer(text))

    # S'il n'y a pas de match, retourne le texte original
    if not matches:
        return [types.Part.from_text(text=text)]

    # Liste pour stocker les segments de texte et les éléments correspondant au pattern
    result = []

    # Position de départ pour le split
    last_end = 0

    # Parcourt tous les matches
    for match in matches:
        # Ajoute le texte avant le match (s'il y en a)
        if match.start() > last_end:
            text_segment = text[last_end : match.start()]
            result.append(types.Part.from_text(text=text_segment))

        # Récupère le chemin de l'image correspondant au match
        img_filename = match.group("filename")
        img_path = os.path.join(image_folder, img_filename)
        try:
            result.append(Image.open(img_path))
        except FileNotFoundError:
            print(f"Image not found: {img_path}")

        # Met à jour la position de fin
        last_end = match.end()

    # Ajoute le reste du texte après le dernier match
    if last_end < len(text):
        result.append(types.Part.from_text(text=text[last_end:]))

    return result

In [ ]:
# Load pompts and tree data
with open(PROJ_ROOT / "config/prompts.json", "r", encoding="utf-8") as f:
    prompts = json.load(f)

prompt_data = []

for page_text in data_zone_uc2["PLU de la Zone UC2"]:
    prompt_data += text_to_part_with_images(
        text=page_text,
        pattern=pattern,
        image_folder=image_folder_zone_uc,
    )

for page_text in data_zone_uc2["Réglement des Zone Urbaines"]:
    prompt_data += text_to_part_with_images(
        text=page_text,
        pattern=pattern,
        image_folder=image_folder_par_zone,
    )

for page_text in data_zone_uc2["Dispositions Générales"]:
    prompt_data += text_to_part_with_images(
        text=page_text,
        pattern=pattern,
        image_folder=image_folder_dg,
    )

prompt_template = get_prompt_grenoble(
    document_content="DOCUMENT_CONTENT",
    zone="Zone UC2",
    prompt=prompts["prompt_grenoble"],
).split("DOCUMENT_CONTENT")

prompt_final = (
    [types.Part.from_text(text=prompt_template[0])]
    + prompt_data
    + [types.Part.from_text(text=prompt_template[1])]
)

In [ ]:
from google import genai

client = genai.Client(
    api_key=os.environ.get("GOOGLE_AI_API_KEY"),
)
model = "gemini-2.5-pro-exp-03-25"
generate_content_config = types.GenerateContentConfig(
    response_mime_type="text/plain",
)
response = client.models.generate_content(
    model=model,
    contents=prompt_final,
    config=generate_content_config,
)

In [ ]:
from typing import Dict, Any, List, Optional


def save_json_data(data: Dict[str, Any], file_path: Path) -> None:
    """
    Sauvegarde des données au format JSON.

    Args:
        data (Dict[str, Any]): Données à sauvegarder
        file_path (Path): Chemin où sauvegarder le fichier
    """
    # Créer le répertoire parent si nécessaire
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


In [ ]:
save_json_data(
    data=response.to_json_dict(), file_path=Path("output/gemini_response/Zone_UC2.json")
)

In [ ]:
import json
import re
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate,
    Paragraph,
    Spacer,
    ListItem,
    ListFlowable,
    Table,
    TableStyle,
)
from reportlab.lib.units import mm


def extract_text_from_json(json_file_path):
    """
    Extrait le contenu textuel de la réponse du modèle à partir du fichier JSON.

    Args:
        json_file_path (str): Chemin vers le fichier JSON

    Returns:
        str: Le contenu textuel extrait
    """
    try:
        # Ouvrir et charger le fichier JSON
        with open(json_file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Naviguer dans la structure JSON pour obtenir le texte
        # Vérifier la structure pour s'assurer que tous les champs existent
        if (
            "candidates" in data
            and len(data["candidates"]) > 0
            and "content" in data["candidates"][0]
            and "parts" in data["candidates"][0]["content"]
            and len(data["candidates"][0]["content"]["parts"]) > 0
            and "text" in data["candidates"][0]["content"]["parts"][0]
        ):
            # Extraire le texte
            text_content = data["candidates"][0]["content"]["parts"][0]["text"]

            # Extraire le contenu Markdown en supprimant les balises de code
            if text_content.startswith("```") and "```" in text_content:
                # Utiliser une expression régulière pour extraire le contenu entre les balises de code
                match = re.search(r"```(?:text|markdown)?\n([\s\S]*?)```", text_content)
                if match:
                    return match.group(1)

            return text_content
        else:
            return "Structure JSON invalide. Impossible de trouver le contenu textuel."

    except json.JSONDecodeError:
        return "Le fichier n'est pas un JSON valide."
    except Exception as e:
        return f"Une erreur est survenue: {str(e)}"


def save_as_markdown(json_file_path, output_file_path):
    """
    Extrait le contenu du JSON et le sauvegarde en format Markdown.

    Args:
        json_file_path (str): Chemin vers le fichier JSON
        output_file_path (str): Chemin où sauvegarder le fichier Markdown

    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        content = extract_text_from_json(json_file_path)

        # Sauvegarder dans un fichier Markdown
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write(content)

        return True
    except Exception as e:
        print(f"Erreur lors de la sauvegarde en Markdown: {str(e)}")
        return False


def format_markdown_text(text):
    """
    Convertit certaines balises Markdown en balises HTML pour ReportLab.

    Args:
        text (str): Texte avec formatage Markdown

    Returns:
        str: Texte avec balises HTML
    """
    # Convertir le texte en gras (**texte**) en balises HTML <b>
    text = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", text)

    # Convertir le texte en italique (*texte*) en balises HTML <i>
    text = re.sub(r"(?<!\*)\*(?!\*)(.*?)(?<!\*)\*(?!\*)", r"<i>\1</i>", text)

    return text


def save_as_pdf(json_file_path, output_file_path):
    """
    Extrait le contenu du JSON et le sauvegarde en format PDF en utilisant ReportLab.

    Args:
        json_file_path (str): Chemin vers le fichier JSON
        output_file_path (str): Chemin où sauvegarder le fichier PDF

    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        content = extract_text_from_json(json_file_path)

        # Créer un nouveau document PDF
        doc = SimpleDocTemplate(
            output_file_path,
            pagesize=A4,
            rightMargin=20 * mm,
            leftMargin=20 * mm,
            topMargin=20 * mm,
            bottomMargin=20 * mm,
        )

        # Définir les styles
        styles = getSampleStyleSheet()

        # Modifier les styles existants au lieu d'en ajouter de nouveaux
        styles["Heading1"].fontSize = 16
        styles["Heading1"].spaceAfter = 10
        styles["Heading1"].spaceBefore = 20
        styles["Heading1"].textColor = colors.darkblue

        styles["Heading2"].fontSize = 14
        styles["Heading2"].spaceAfter = 8
        styles["Heading2"].spaceBefore = 16
        styles["Heading2"].textColor = colors.darkblue

        styles["Heading3"].fontSize = 12
        styles["Heading3"].spaceAfter = 6
        styles["Heading3"].spaceBefore = 12
        styles["Heading3"].textColor = colors.darkblue

        # Vérifier si Heading4 existe et le modifier ou le créer
        if "Heading4" in styles:
            styles["Heading4"].fontSize = 11
            styles["Heading4"].spaceAfter = 4
            styles["Heading4"].spaceBefore = 10
            styles["Heading4"].textColor = colors.darkblue
        else:
            styles.add(
                ParagraphStyle(
                    name="Heading4",
                    parent=styles["Normal"],
                    fontSize=11,
                    spaceAfter=4,
                    spaceBefore=10,
                    textColor=colors.darkblue,
                )
            )

        # Vérifier si Heading5 existe et le modifier ou le créer
        if "Heading5" in styles:
            styles["Heading5"].fontSize = 10
            styles["Heading5"].spaceAfter = 4
            styles["Heading5"].spaceBefore = 8
            styles["Heading5"].textColor = colors.darkblue
        else:
            styles.add(
                ParagraphStyle(
                    name="Heading5",
                    parent=styles["Normal"],
                    fontSize=10,
                    spaceAfter=4,
                    spaceBefore=8,
                    textColor=colors.darkblue,
                )
            )

        # Modifier le style Normal
        styles["Normal"].fontSize = 10
        styles["Normal"].spaceAfter = 6

        # Vérifier si ListItem existe et le modifier ou le créer
        if "ListItem" in styles:
            styles["ListItem"].leftIndent = 20
            styles["ListItem"].spaceAfter = 3
        else:
            styles.add(
                ParagraphStyle(
                    name="ListItem",
                    parent=styles["Normal"],
                    leftIndent=20,
                    spaceAfter=3,
                )
            )

        # Créer le style Source (il n'existe pas par défaut)
        styles.add(
            ParagraphStyle(
                name="Source",
                parent=styles["Normal"],
                fontSize=8,
                textColor=colors.grey,
                spaceAfter=6,
                leftIndent=20,
            )
        )

        # Préparer la liste des éléments à ajouter au PDF
        elements = []

        # Traiter le contenu ligne par ligne
        lines = content.split("\n")
        in_list = False
        list_items = []
        current_list_type = None  # 'bullet' ou 'numbered'

        for line in lines:
            # Traiter les en-têtes
            if line.startswith("# "):
                if in_list:
                    # Ajouter la liste précédente si on entre dans un nouvel élément
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[2:]), styles["Heading1"])
                )
            elif line.startswith("## "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[3:]), styles["Heading2"])
                )
            elif line.startswith("### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[4:]), styles["Heading3"])
                )
            elif line.startswith("#### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[5:]), styles["Heading4"])
                )
            elif line.startswith("##### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[6:]), styles["Heading5"])
                )

            # Traiter les éléments de liste
            elif line.strip().startswith("* ") or line.strip().startswith("- "):
                in_list = True
                current_list_type = "bullet"
                text = line.strip()[2:]

                # Vérifier s'il s'agit d'une source
                if "(Source:" in text:
                    parts = text.split("(Source:", 1)
                    content_text = format_markdown_text(parts[0].strip())
                    source_text = "(Source:" + parts[1]
                    list_items.append(
                        ListItem(Paragraph(content_text, styles["ListItem"]))
                    )
                    list_items.append(
                        ListItem(Paragraph(source_text, styles["Source"]))
                    )
                else:
                    list_items.append(
                        ListItem(
                            Paragraph(format_markdown_text(text), styles["ListItem"])
                        )
                    )

            # Traiter les paragraphes normaux
            elif line.strip():
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None

                # Vérifier s'il s'agit d'une source
                if "(Source:" in line:
                    parts = line.split("(Source:", 1)
                    content_text = format_markdown_text(parts[0].strip())
                    source_text = "(Source:" + parts[1]
                    if content_text:
                        elements.append(Paragraph(content_text, styles["Normal"]))
                    elements.append(Paragraph(source_text, styles["Source"]))
                else:
                    elements.append(
                        Paragraph(format_markdown_text(line), styles["Normal"])
                    )

            # Ajouter un espace pour les lignes vides
            elif not line.strip() and not in_list:
                elements.append(Spacer(1, 3 * mm))

        # Ajouter la dernière liste si elle existe
        if in_list and list_items:
            elements.append(
                ListFlowable(list_items, bulletType=current_list_type or "bullet")
            )

        # Générer le PDF
        doc.build(elements)

        return True
    except Exception as e:
        print(f"Erreur lors de la conversion en PDF avec ReportLab: {str(e)}")
        return False


json_file = "/mnt/mydisk/Projects/plu/notebooks/output/gemini_response/Zone_UC2.json"
save_as_markdown(json_file, "Zone_UC2.md")
save_as_pdf(json_file, "Zone_UC2.pdf")
print("Conversion terminée!")

## Completion of the output

Complétion de l'analyse de la zone avec les documents de l'OAP.

In [ ]:
import re
from PIL import Image
import os

with open("/mnt/mydisk/Projects/plu/config/prompts.json", "r", encoding="utf-8") as f:
    prompt_oap = json.load(f)["prompt_oap"]

with open("output/gemini_response/Zone_UC2.json", "r", encoding="utf-8") as f:
    response_data = json.load(f)

# OAP
with open(
    "/mnt/mydisk/Projects/plu/data/raw/Grenoble/Carnet de Paysage.json",
    "r",
    encoding="utf-8",
) as f:
    data_paysage = json.load(f)

with open(
    "/mnt/mydisk/Projects/plu/data/raw/Grenoble/Plan de Prévention du Risque Inondation.json",
    "r",
    encoding="utf-8",
) as f:
    data_ppri = json.load(f)

image_folder_paysage = (
    "/mnt/mydisk/Projects/plu/data/raw/markdown/Grenoble/Carnet de Paysage/images"
)
image_folder_ppri = "/mnt/mydisk/Projects/plu/data/raw/markdown/Grenoble/Plan de Prévention du Risque Inondation/images"

pattern = r"!\[(?P<alt>[^\]]*)\]\((?P<filename>[^)]+)\)"

In [ ]:
filtered_paysage = [
    f"page {page['index']}: {page['markdown']}" for page in data_paysage["pages"]
]
filtered_ppri = [
    f"page {page['index']}: {page['markdown']}" for page in data_ppri["pages"]
]

In [ ]:
prompt_data_s2 = []

for page_text in filtered_paysage:
    prompt_data_s2 += text_to_part_with_images(
        text=page_text,
        pattern=pattern,
        image_folder=image_folder_paysage,
    )

for page_text in filtered_ppri:
    prompt_data_s2 += text_to_part_with_images(
        text=page_text,
        pattern=pattern,
        image_folder=image_folder_ppri,
    )

In [ ]:
def get_prompt_oap(
    zone: str,
    prompt: str,
) -> str:
    """
    Récupère le prompt pour la ville de Grenoble.

    Args:
        data_general: (dict) Les données générales de la ville de Grenoble.
        data_par_zone: (dict) Les données par zone de la ville de Grenoble.
        data_zone: (dict) Les données de zone de la ville de Grenoble.
        type_zone: (str) Le type de zone à analyser.
        zone: (str) La zone à analyser.
        prompt: (str) Le prompt de base.

    Returns:
        str: Le prompt pour la ville de Grenoble.
    """
    return prompt.format(
        ANALYSE_PRELIMINAIRE_A_PEAUFINER="ANALYSE_PRELIMINAIRE_A_PEAUFINER",
        NOUVEAUX_DOCUMENTS_OAP="NOUVEAUX_DOCUMENTS_OAP",
        ZONE_CADASTRALE_CIBLE=zone,
    )

In [ ]:
prompt_template_oap = get_prompt_oap(
    zone="Zone UC2",
    prompt=prompt_oap,
).split("NOUVEAUX_DOCUMENTS_OAP")
prompt_final_oap = (
    [types.Part.from_text(text=prompt_template_oap[0].split("ANALYSE_PRELIMINAIRE_A_PEAUFINER")[0])]
    + [types.Part.from_text(text=response_data["candidates"][0]["content"]["parts"][0]["text"])]
    + [types.Part.from_text(text=prompt_template_oap[0].split("ANALYSE_PRELIMINAIRE_A_PEAUFINER")[1])]
    + prompt_data_s2
    + [types.Part.from_text(text=prompt_template_oap[1])]
)

In [ ]:
from google import genai

client = genai.Client(
    api_key=os.environ.get("GOOGLE_AI_API_KEY"),
)
model = "gemini-2.5-pro-exp-03-25"
generate_content_config = types.GenerateContentConfig(
    response_mime_type="text/plain",
)
response = client.models.generate_content(
    model=model,
    contents=prompt_final_oap,
    config=generate_content_config,
)

In [ ]:
save_json_data(
    data=response.to_json_dict(), file_path=Path("output/gemini_response/Zone_UC2_et_oap.json")
)

In [ ]:
import json
import re
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate,
    Paragraph,
    Spacer,
    ListItem,
    ListFlowable,
    Table,
    TableStyle,
)
from reportlab.lib.units import mm


def extract_text_from_json(json_file_path):
    """
    Extrait le contenu textuel de la réponse du modèle à partir du fichier JSON.

    Args:
        json_file_path (str): Chemin vers le fichier JSON

    Returns:
        str: Le contenu textuel extrait
    """
    try:
        # Ouvrir et charger le fichier JSON
        with open(json_file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # Naviguer dans la structure JSON pour obtenir le texte
        # Vérifier la structure pour s'assurer que tous les champs existent
        if (
            "candidates" in data
            and len(data["candidates"]) > 0
            and "content" in data["candidates"][0]
            and "parts" in data["candidates"][0]["content"]
            and len(data["candidates"][0]["content"]["parts"]) > 0
            and "text" in data["candidates"][0]["content"]["parts"][0]
        ):
            # Extraire le texte
            text_content = data["candidates"][0]["content"]["parts"][0]["text"]

            # Extraire le contenu Markdown en supprimant les balises de code
            if text_content.startswith("```") and "```" in text_content:
                # Utiliser une expression régulière pour extraire le contenu entre les balises de code
                match = re.search(r"```(?:text|markdown)?\n([\s\S]*?)```", text_content)
                if match:
                    return match.group(1)

            return text_content
        else:
            return "Structure JSON invalide. Impossible de trouver le contenu textuel."

    except json.JSONDecodeError:
        return "Le fichier n'est pas un JSON valide."
    except Exception as e:
        return f"Une erreur est survenue: {str(e)}"


def save_as_markdown(json_file_path, output_file_path):
    """
    Extrait le contenu du JSON et le sauvegarde en format Markdown.

    Args:
        json_file_path (str): Chemin vers le fichier JSON
        output_file_path (str): Chemin où sauvegarder le fichier Markdown

    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        content = extract_text_from_json(json_file_path)

        # Sauvegarder dans un fichier Markdown
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write(content)

        return True
    except Exception as e:
        print(f"Erreur lors de la sauvegarde en Markdown: {str(e)}")
        return False


def format_markdown_text(text):
    """
    Convertit certaines balises Markdown en balises HTML pour ReportLab.

    Args:
        text (str): Texte avec formatage Markdown

    Returns:
        str: Texte avec balises HTML
    """
    # Convertir le texte en gras (**texte**) en balises HTML <b>
    text = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", text)

    # Convertir le texte en italique (*texte*) en balises HTML <i>
    text = re.sub(r"(?<!\*)\*(?!\*)(.*?)(?<!\*)\*(?!\*)", r"<i>\1</i>", text)

    return text


def save_as_pdf(json_file_path, output_file_path):
    """
    Extrait le contenu du JSON et le sauvegarde en format PDF en utilisant ReportLab.

    Args:
        json_file_path (str): Chemin vers le fichier JSON
        output_file_path (str): Chemin où sauvegarder le fichier PDF

    Returns:
        bool: True si l'opération a réussi, False sinon
    """
    try:
        content = extract_text_from_json(json_file_path)

        # Créer un nouveau document PDF
        doc = SimpleDocTemplate(
            output_file_path,
            pagesize=A4,
            rightMargin=20 * mm,
            leftMargin=20 * mm,
            topMargin=20 * mm,
            bottomMargin=20 * mm,
        )

        # Définir les styles
        styles = getSampleStyleSheet()

        # Modifier les styles existants au lieu d'en ajouter de nouveaux
        styles["Heading1"].fontSize = 16
        styles["Heading1"].spaceAfter = 10
        styles["Heading1"].spaceBefore = 20
        styles["Heading1"].textColor = colors.darkblue

        styles["Heading2"].fontSize = 14
        styles["Heading2"].spaceAfter = 8
        styles["Heading2"].spaceBefore = 16
        styles["Heading2"].textColor = colors.darkblue

        styles["Heading3"].fontSize = 12
        styles["Heading3"].spaceAfter = 6
        styles["Heading3"].spaceBefore = 12
        styles["Heading3"].textColor = colors.darkblue

        # Vérifier si Heading4 existe et le modifier ou le créer
        if "Heading4" in styles:
            styles["Heading4"].fontSize = 11
            styles["Heading4"].spaceAfter = 4
            styles["Heading4"].spaceBefore = 10
            styles["Heading4"].textColor = colors.darkblue
        else:
            styles.add(
                ParagraphStyle(
                    name="Heading4",
                    parent=styles["Normal"],
                    fontSize=11,
                    spaceAfter=4,
                    spaceBefore=10,
                    textColor=colors.darkblue,
                )
            )

        # Vérifier si Heading5 existe et le modifier ou le créer
        if "Heading5" in styles:
            styles["Heading5"].fontSize = 10
            styles["Heading5"].spaceAfter = 4
            styles["Heading5"].spaceBefore = 8
            styles["Heading5"].textColor = colors.darkblue
        else:
            styles.add(
                ParagraphStyle(
                    name="Heading5",
                    parent=styles["Normal"],
                    fontSize=10,
                    spaceAfter=4,
                    spaceBefore=8,
                    textColor=colors.darkblue,
                )
            )

        # Modifier le style Normal
        styles["Normal"].fontSize = 10
        styles["Normal"].spaceAfter = 6

        # Vérifier si ListItem existe et le modifier ou le créer
        if "ListItem" in styles:
            styles["ListItem"].leftIndent = 20
            styles["ListItem"].spaceAfter = 3
        else:
            styles.add(
                ParagraphStyle(
                    name="ListItem",
                    parent=styles["Normal"],
                    leftIndent=20,
                    spaceAfter=3,
                )
            )

        # Créer le style Source (il n'existe pas par défaut)
        styles.add(
            ParagraphStyle(
                name="Source",
                parent=styles["Normal"],
                fontSize=8,
                textColor=colors.grey,
                spaceAfter=6,
                leftIndent=20,
            )
        )

        # Préparer la liste des éléments à ajouter au PDF
        elements = []

        # Traiter le contenu ligne par ligne
        lines = content.split("\n")
        in_list = False
        list_items = []
        current_list_type = None  # 'bullet' ou 'numbered'

        for line in lines:
            # Traiter les en-têtes
            if line.startswith("# "):
                if in_list:
                    # Ajouter la liste précédente si on entre dans un nouvel élément
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[2:]), styles["Heading1"])
                )
            elif line.startswith("## "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[3:]), styles["Heading2"])
                )
            elif line.startswith("### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[4:]), styles["Heading3"])
                )
            elif line.startswith("#### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[5:]), styles["Heading4"])
                )
            elif line.startswith("##### "):
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None
                elements.append(
                    Paragraph(format_markdown_text(line[6:]), styles["Heading5"])
                )

            # Traiter les éléments de liste
            elif line.strip().startswith("* ") or line.strip().startswith("- "):
                in_list = True
                current_list_type = "bullet"
                text = line.strip()[2:]

                # Vérifier s'il s'agit d'une source
                if "(Source:" in text:
                    parts = text.split("(Source:", 1)
                    content_text = format_markdown_text(parts[0].strip())
                    source_text = "(Source:" + parts[1]
                    list_items.append(
                        ListItem(Paragraph(content_text, styles["ListItem"]))
                    )
                    list_items.append(
                        ListItem(Paragraph(source_text, styles["Source"]))
                    )
                else:
                    list_items.append(
                        ListItem(
                            Paragraph(format_markdown_text(text), styles["ListItem"])
                        )
                    )

            # Traiter les paragraphes normaux
            elif line.strip():
                if in_list:
                    if list_items:
                        elements.append(
                            ListFlowable(
                                list_items, bulletType=current_list_type or "bullet"
                            )
                        )
                        list_items = []
                        in_list = False
                        current_list_type = None

                # Vérifier s'il s'agit d'une source
                if "(Source:" in line:
                    parts = line.split("(Source:", 1)
                    content_text = format_markdown_text(parts[0].strip())
                    source_text = "(Source:" + parts[1]
                    if content_text:
                        elements.append(Paragraph(content_text, styles["Normal"]))
                    elements.append(Paragraph(source_text, styles["Source"]))
                else:
                    elements.append(
                        Paragraph(format_markdown_text(line), styles["Normal"])
                    )

            # Ajouter un espace pour les lignes vides
            elif not line.strip() and not in_list:
                elements.append(Spacer(1, 3 * mm))

        # Ajouter la dernière liste si elle existe
        if in_list and list_items:
            elements.append(
                ListFlowable(list_items, bulletType=current_list_type or "bullet")
            )

        # Générer le PDF
        doc.build(elements)

        return True
    except Exception as e:
        print(f"Erreur lors de la conversion en PDF avec ReportLab: {str(e)}")
        return False


json_file = "/mnt/mydisk/Projects/plu/notebooks/output/gemini_response/Zone_UC2_et_oap.json"
save_as_markdown(json_file, "Zone_UC2_et_oap.md")
save_as_pdf(json_file, "Zone_UC2_et_oap.pdf")
print("Conversion terminée!")